In [1]:
# %% test cuda
import torch #type: ignore
torch.cuda.is_available()

True

In [1]:
# %% test a specific model
from run_model import main
main(process = 0 , resume = 0 , checkname= 1)

Use device name: NVIDIA GeForce RTX 4090


24-04-14 18:33:45|MOD:run_model   |: Model Specifics:
24-04-14 18:33:45|MOD:run_model   |: Start Process [Load Data]!


--Process Queue : Data + Train + Test
--Start Training New!
--Model_name is set to patch_tst_day.2!
{'random_seed': None,
 'model_name': 'patch_tst_day.2',
 'model_module': 'patch_tst',
 'model_data_type': 'day',
 'model_data_prenorm': {'day': {'divlast': True, 'histnorm': True}},
 'labels': ['std_lag1_10'],
 'beg_date': 20170103,
 'end_date': 99991231,
 'interval': 120,
 'input_step_day': 5,
 'test_step_day': 1}
{'hidden_dim': [32],
 'seqlens': [{'day': 30, '30m': 30, 'dms': 30}],
 'seq_len': [30],
 'dropout': [0.1],
 'attn_dropout': [0.1],
 'num_output': [1],
 'shared_embedding': [True],
 'shared_head': [True, False],
 'revin': [True],
 'act_type': ['gelu'],
 'patch_len': [5],
 'stride': [2],
 'n_layers': [2],
 'n_heads': [4],
 'd_ff': [64]}
use /home/mengkjin/Workspace/learndl/data/torch_pack/day.20240313.pt


24-04-14 18:33:48|MOD:run_model   |: Finish Process [Load Data]! Cost 3.2Secs
24-04-14 18:33:48|MOD:run_model   |: Start Process [Train Model]!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


patch_tst_day.2 #0 @20170103 LoadData Cost    3.0Secs


score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


FirstBite Ep#  0 : loss  0.97500, train 0.04156, valid 0.07322, max 0.0732, best 0.0732, lr1.0e-07
FirstBite Ep#  5 : loss  0.89137, train 0.12619, valid 0.14235, max 0.1424, best 0.1424, lr3.8e-03
FirstBite Ep# 10 : loss  0.87043, train 0.14822, valid 0.13979, max 0.1424, best 0.1424, lr1.3e-03
FirstBite Ep# 15 : loss  0.85571, train 0.16368, valid 0.13305, max 0.1424, best 0.1424, lr6.3e-04
FirstBite Ep# 20 : loss  0.85302, train 0.16645, valid 0.13477, max 0.1424, best 0.1424, lr1.3e-03
FirstBite Ep# 25 : loss  0.84473, train 0.17479, valid 0.13078, max 0.1424, best 0.1424, lr1.6e-04
24-04-14 18:36:54|MOD:run_model   |: patch_tst_day.2 #0 @20170103|FirstBite Ep# 25 EarlyStop|Train 0.1748 Valid 0.1308 BestVal 0.1424|Cost  3.1Min,  7.0Sec/Ep
FirstBite Ep#  0 : loss  1.02970, train-0.03972, valid-0.08545, max-0.0854, best-0.0854, lr1.0e-07
FirstBite Ep#  5 : loss  0.89242, train 0.12521, valid 0.13678, max 0.1368, best 0.1368, lr3.8e-03
FirstBite Ep# 10 : loss  0.86739, train 0.15161, 

In [1]:
from src.data import PreProcess
PreProcess.main(False)
PreProcess.main(True)

Use device name: NVIDIA GeForce RTX 4090


24-03-31 09:52:11|MOD:data_preprocessing|: if_train is False , Data Processing start!
24-03-31 09:52:11|MOD:data_preprocessing|: 3 datas :['y', 'trade_day', 'trade_30m']


Sun Mar 31 09:52:11 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 0.39 secs
labels blocks reading ret20_lag DataBase's ...... cost 0.33 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_exp DataBase's ...... cost 1.66 secs
models blocks merging ...... cost 0.17 secs
y blocks process ...... cost 0.88 secs
y blocks masking ...... cost 0.05 secs
y blocks saving  ...... cost 0.14 secs
y blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:14 2024 : y finished! Cost 3.67 Seconds
Sun Mar 31 09:52:14 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 0.56 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.03 secs
trade_day blocks masking ...... cost 0.05 secs
trade_day blocks saving  ...... cost 0.16 secs
trade_day blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:15 2024 : trade_day finished! Cost 0.81 Seconds
Sun Mar 31 09:52:15 2024 : trade_30m start ...
trade blocks r

24-03-31 09:52:21|MOD:data_preprocessing|: Data Processing Finished! Cost 10.02 Seconds


... cost 1.13 secs
trade_30m blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:21 2024 : trade_30m finished! Cost 5.19 Seconds
